### PAS MAL

In [272]:
# importer les librairies nécessaires
import os, re, glob, csv
from spellchecker import SpellChecker
from lxml import etree
    

# ignorer les fichiers cachés dans le directoire avec les docs d'entrée (p. ex. le '._5419000_r.xml') 
def listdir_nohidden(path):
    return glob.glob(os.path.join(path, '*'))


# spécifier les docs d'entrée (échantillon) à partir desquels les sorties corrigées seront générées
directory_in = listdir_nohidden("./sample_in/")


# enlever l'extension .xml des fichiers d'entrée 
for file_in in directory_in:
    tree = etree.parse(file_in)
    root = tree.getroot()
    file_in = os.path.basename(file_in)
    file_in = os.path.splitext(file_in)[0]
    # print(file_in) # 5419000_r, test
    
    
    # créer les nouveaux fichiers .txt sur lesquels les corrections seront appliquées par la suite
    file_out = '{}'.format(file_in)+'.txt'
    # print(file_out) # 5419000_r.txt, test.txt
    directory_out = os.path.join("./sample_out/", file_out)
  
    
    # créer les nouveaux fichiers .csv où les erreurs avec leurs corrections seront enregistrées
    corr_out = os.path.join('./csv/', file_in+'.csv')
    
    
    # transférer le contenu des fichiers .xml dans les fichiers .txt
    # nettoyer les .txt
    car_spec = ['&', '■', '•', '&', '%', '*', '#', '+', '^', '\\', '$', '>', '<', '|', '£']
    with open(directory_out, 'w') as f, open(corr_out, 'w') as fout:
        writer = csv.writer(fout)
        writer.writerow(["Erreur"'\t' "Correction"'\t' "Fréquence"'\t'])
        for elem in root.iter('*'):
            if elem.text is not None:
                text = elem.text.strip()
                if text: 
                    for c in car_spec:
                        text = text.replace(c,'')
                    text = re.sub('« \n', '', text)
                    text = re.sub(" +", " ", text)
                    text = text.lower()
                    text = text.replace("\n", " ")
                    text = text.replace(' ,', ',')
                    text = text.replace("'", "’")
                    text = text.replace(' .','. ')
                    text = text.replace(' :',':')
                    text = text.replace(' ;',';')
                    text = text.replace(' !','!')
                    text = text.replace(' ?','?')
                    text = text.replace('" ','"')
                    text = text.replace('( ','(')
                    text = text.replace(' )',')') 
                            

                    # définir le correcteur d'orthographe français 
                    # pyspellchecker
                    spell = SpellChecker(language='fr')

                    
                    # tokeniser le texte avec le tokeniseur standard (ex: 'l'empire')
                    # car celui de pyspellchecker tokenise mal (ex: 'l', 'empire')
                    token_list = text.split()
                    # tokeniser le texte avec le tokeniseur de pyspellchecker
                    # tokens = spell.split_words(text)
                    # print(content_list)                    


                    # ne pas corriger les tokens contenant l'apostrophe (ex : l’empire, d’art, s’étend...)
                    r1 = re.findall(r"(l’\w+|l’\w+-\w+|d’\w+|d’\w+|qu’\w+|c’\w+|n’\w+|j’\w+|lorfqu’\w+|eft)",text)
                    spell.word_frequency.load_words(r1)
                    spell.known(r1)  # les mots {'ex : l’empire, d’art, s’étend'} sont désormais dans le dictionnaire des mots corrects

                    
                    # corriger les mots incorrects dans le fichier .txt
                    # extraire les erreurs et leurs corrections dans un tableur .csv
                    misspelled = spell.unknown(token_list)
                    for m in misspelled:
                        corrected = spell.correction(m)
                        text = text.replace(m, corrected)
                        # f.write(c.replace('clafliques', 'classiques'))
                        fout.write(m+'\t' + corrected+'\t' + ' \n')
                    # print(text)
                    f.write(text + "\n")

### confucrus, prehier...

In [145]:
import re, glob
from spellchecker import SpellChecker

entry = "5419000_r.txt"
output = "5419000_r_corr_lower.txt"


# activer le correcteur d'orthographe français
spell = SpellChecker(language='fr')
spell.distance = 2



text = open(entry).read()
text_lower = text.lower()

# ne pas corriger les tokens contenant l'apostrophe (ex : l’empire, d’art, s’étend...)
r1 = re.findall(r"([lL]’\w+|[dD]’\w+|[sS]’\w+|[qQ]u’\w+|[cC]’\w+|[nN]’\w+|[jJ]’\w+|[Ll]orfqu’\w+|eft)",text_lower)


# tokeniser le texte avec le tokeniseur de pyspellchecker
tokens = spell.split_words(text_lower)


spell.word_frequency.load_words(r1)
spell.known(r1)  # les mots {'ex : l’empire, d’art, s’étend'} sont désormais dans le dictionnaire des mots corrects




misspelled = spell.unknown(tokens)


with open(output, "w") as f:
    for m in misspelled:
        corrected = spell.correction(m)
        text_lower = text_lower.replace(m, corrected)
    print(text_lower)
        # f.write(c.replace('clafliques', 'classiques'))
    f.write(text_lower)
    

FileNotFoundError: [Errno 2] No such file or directory: '5419000_r.txt'

### Nettoyage

In [19]:
from lxml import etree
import re

tree = etree.parse('5419000_r.xml')
root = tree.getroot()

with open('5419000_r.txt', 'w') as f:
    for elem in root.iter('*'):
        if elem.text is not None:
            text = elem.text.strip()
            if text: 
                text = text.replace("\n", " ")
                text = re.sub(" +", " ", text)
                print(text)
                f.write(text + "\n")
                

Les livres classiques de l’empire de la Chine
Confucius (0551?-0479? av. J.-C.)
Annotateur
Pluquet, François-André-Adrien (1716-1790)
Traducteur
Noël, François (1651-1729)
TGB (BnF – OBVIL)
Les livres classiques de l’empire de la Chine
Tome 2
cb30263946g
http://gallica.bnf.fr/ark:/12148/bpt6k54190001
Barrois aîné et Barrois jeune
1784
1784
Les livres classiques de l’empire de la Chine — Tome 2
950 - Histoire générale de l’Asie
0.67
confucius
1233
Confucius
11909957
Confucius (0551?-0479? av. J.-C.)
True
0479
Auteur du texte
0551
Livres classiques
DE L’EMPIRE .
De la chine.
Observations
SUR
LES LIVRES CLASSIQUES
DE L’EMPIRE
DE LA CHINE.
.LES Chinois ont deux sortes de livres clafliques ou canoniques : les Kings, ou les livres canoniques du premier ordre ; & les Ssée-chu, ou livres canoniques dusecond ordre.
Les Kings sont au nombre de cinq ; l’Y-king, le Chu-king,lc Chi-king, le Tchun-tfiou & le Lild.
L’Y-king remonte à la plus haute
Tome II. a
’\
antiquité ; on l’attribue en grande par

aucune mauvaife affeétion, c’eft ce que l’on nomme l’amour parfait, ou la doitiue abfolue du coeur.
Donner aux autres un degré de perfection qui les dirige confram-* ment & sûrement félon les règles
o
de l’honnêteté, c’eft ce que l’on appelle la prudence.
L’homme reçoit ces deux vertus de la nature ; & leur manière d’agir au dedans ou au dehors eft la même : ainfi celui qui les pofsede fait toujours bien ce qu’il fait, & suit toujours la vérité.
La parfaite vertu agit toujours, elle persévère long-temps, s’affermit, & jette de profondes racines
I5>0 LE JUSTE MILIEU.
dans le coeur: iorfcju’elleapersévéré & jette de profondes racines dans le coeur, elle fe manifefte, elle fe répand au loin , remplit toutes les parties du monde par Ton étendue, & pénètre dans tous les coeurs par fa profondeur : c’eft alors qu’elle eft parvenue au plus haut degré de fon entreprife, & que fa perfection eft dans fa plus grande fplendeur. Son étendue & fa profondeur foutiennenttous fesouvrages, fon élévation 

In [235]:
for m in misspelled:
    corrected = spell.correction('« être')
print(corrected)

a


In [ ]:
# importer les librairies nécessaires
import regex as re
import csv
from spellchecker import SpellChecker


# # spécifier les docs d'entrés à corriger et ceux de sortie qui seront corrigés
# entry = "sample.xml"
# output = "sample_out.xml"

# ignorer les fichiers cachés dans le directoire avec les docs d'entrée (p. ex. le '._5419000_r.xml') 
def listdir_nohidden(path):
    return glob.glob(os.path.join(path, '*'))


# spécifier les docs d'entrée (échantillon) à partir desquels les sorties corrigées seront générées
directory_in = listdir_nohidden("./sample_in/")
directory_out = listdir_nohidden("./sample_out/")


# enlever les balises XML
text = re.sub('<[^<]+>', "", open(entry).read()) 
with open(output, "w") as f:
    f.write(text)
    

# enlever les espaces dans le fichier XML    
text = re.sub('\s+',' ', open(output).read())

    
# enlever les signes de ponctuation sauf les apostrophes, les points d’interrogations, les parenthèses et les tirets
text = re.sub(r"""[!^\.\,\!\*]+""",'',text)
with open(output, "w") as f:
    f.write(text)  
    
    
# récupérer la liste des tokens   
my_file = open(output, "r")
content = my_file.read()
# print(content) # imprimer le contenu

content_list = content.split(" ")
my_file.close()
# print(content_list) # imprimer la liste des tokens


#  extraire les tokens mal orthographiés et leurs corrections
fout = open("corrections.csv", "wt")
spell = SpellChecker(language='fr')
misspelled = spell.unknown(content_list)
writer = csv.writer(fout)
writer.writerow(["Erreur"'\t' "Correction"'\t' "Fréquence"'\t'])
# print(misspelled) # imprimer les tokens mal orthographiés
for word in misspelled:
	fout.write(word+'\t' + spell.correction(word)+'\t' + ' \n')
	

fout.close()

In [147]:
# importer les librairies nécessaires
import os, re, glob
from spellchecker import SpellChecker
from lxml import etree
    

# ignorer les fichiers cachés dans le directoire avec les docs d'entrée (p. ex. le '._5419000_r.xml') 
def listdir_nohidden(path):
    return glob.glob(os.path.join(path, '*'))


# spécifier les docs d'entrée (échantillon) à partir desquels les sorties corrigées seront générées
directory_in = listdir_nohidden("./sample_in/")


# enlever l'extension .xml des fichiers d'entrée 
for file_in in directory_in:
    tree = etree.parse(file_in)
    root = tree.getroot()
    file_in = os.path.basename(file_in)
    file_in = os.path.splitext(file_in)[0]

    # print(file_in) # 5419000_r, bla
    
    
    # créer les nouveaux fichiers .txt sur lesquels les corrections seront appliquées par la suite
    file_out = '{}'.format(file_in)+'.txt'
    # print(file_out) # 5419000_r.txt, bla.txt
    directory_out = os.path.join("./sample_out/", file_out)
    # print(directory_out)
    
    
    # transférer le contenu des fichiers .xml dans les fichiers .txt
    # nettoyer les .txt
    with open(directory_out, 'w') as f:
        for elem in root.iter('*'):
            if elem.text is not None:
                text = elem.text.strip()
                if text: 
                    text = text.lower()
                    text = text.replace("\n", " ")
                    text = text.replace("'", "’")
                    text = re.sub(" +", " ", text)
                    # print(text)
                    

                    


                    # définir le correcteur d'orthographe français 
                    # pyspellchecker
                    spell = SpellChecker(language='fr')

                    # tokeniser le texte avec le tokeniseur standard
                    token_list = text.split()
                    # tokeniser le texte avec le tokeniseur de pyspellchecker
                    # tokens = spell.split_words(text)
                    # print(content_list)                    


                    # ne pas corriger les tokens contenant l'apostrophe (ex : l’empire, d’art, s’étend...)
                    r1 = re.findall(r"(l’\w+|l’\w+-\w+|d’\w+|d’\w+|qu’\w+|c’\w+|n’\w+|j’\w+|lorfqu’\w+|eft)",text)

                    spell.word_frequency.load_words(r1)
                    spell.known(r1)  # les mots {'ex : l’empire, d’art, s’étend'} sont désormais dans le dictionnaire des mots corrects

                    misspelled = spell.unknown(token_list)
                        
    

                    for m in misspelled:
                        corrected = spell.correction(m)
                        text = text.replace(m, corrected)
                        # f.write(c.replace('clafliques', 'classiques'))
                    f.write(text + "\n")

In [180]:
# importer les librairies nécessaires
import os, re, glob
from spellchecker import SpellChecker
from lxml import etree
    

# ignorer les fichiers cachés dans le directoire avec les docs d'entrée (p. ex. le '._5419000_r.xml') 
def listdir_nohidden(path):
    return glob.glob(os.path.join(path, '*'))


# spécifier les docs d'entrée (échantillon) à partir desquels les sorties corrigées seront générées
directory_in = listdir_nohidden("./sample_in/")


# enlever l'extension .xml des fichiers d'entrée 
for file_in in directory_in:
    tree = etree.parse(file_in)
    root = tree.getroot()
    file_in = os.path.basename(file_in)
    file_in = os.path.splitext(file_in)[0]

    # print(file_in) # 5419000_r, bla
    
    
    # créer les nouveaux fichiers .txt sur lesquels les corrections seront appliquées par la suite
    file_out = '{}'.format(file_in)+'.txt'
    # print(file_out) # 5419000_r.txt, bla.txt
    directory_out = os.path.join("./sample_out/", file_out)
    # print(directory_out)
    
    
    # transférer le contenu des fichiers .xml dans les fichiers .txt
    # nettoyer les .txt
    with open(directory_out, 'w') as f:
        for elem in root.iter('*'):
            if elem.text is not None:
                text = elem.text.strip()
                if text: 
                    text = re.sub(" +", " ", text)
                    text = text.lower()
                    text = text.replace("\n", " ")
                    text = text.replace(' ,', ',')
                    text = text.replace("'", "’")
                    text = text.replace(' .','. ')
                    text = text.replace(' :',':')
                    text = text.replace(' ;',';')
                    text = text.replace(' !','!')
                    text = text.replace(' ?','?')
                    text = text.replace("&", "")
                    # text = re.sub(r'\W+', ' ', text)
                    
                    print(text)

les livres classiques de l empire de la chine
confucius 0551 0479 av j c 
annotateur
pluquet françois andré adrien 1716 1790 
traducteur
noël françois 1651 1729 
tgb bnf obvil 
les livres classiques de l empire de la chine
tome 2
cb30263946g
http gallica bnf fr ark 12148 bpt6k54190001
barrois aîné et barrois jeune
1784
1784
les livres classiques de l empire de la chine tome 2
950 histoire générale de l asie
0 67
confucius
1233
confucius
11909957
confucius 0551 0479 av j c 
true
0479
auteur du texte
0551
livres classiques
de l empire 
de la chine 
observations
sur
les livres classiques
de l empire
de la chine 
 les chinois ont deux sortes de livres clafliques ou canoniques les kings ou les livres canoniques du premier ordre les ssée chu ou livres canoniques dusecond ordre 
les kings sont au nombre de cinq l y king le chu king lc chi king le tchun tfiou le lild 
l y king remonte à la plus haute
tome ii a
 
antiquité on l attribue en grande partie à fo hi c eft un ouvrage qui par le moyen

### Correction orthographique<br>
`5_out.txt` : `avet` au lieu de `avec` (prb)

In [ ]:
import re, glob
from spellchecker import SpellChecker

entry = "1.txt"
output = "1_corr.txt"


# activer le correcteur d'orthographe français
spell = SpellChecker(language='fr')


text = open(entry).read()


# ne pas corriger les tokens contenant l'apostrophe (ex : l’empire, d’art, s’étend...)
r1 = re.findall(r"([lL]’\w+|[dD]’\w+|[sS]’\w+|[qQ]u’\w+|[cC]’\w+|[nN]’\w+|[jJ]’\w+|[Ll]orfqu’\w+|eft)",text)


# tokeniser le texte avec le tokeniseur de pyspellchecker
tokens = spell.split_words(text)


spell.word_frequency.load_words(r1)
spell.known(r1)  # les mots {'ex : l’empire, d’art, s’étend'} sont désormais dans le dictionnaire des mots corrects

print(tokens)

misspelled = spell.unknown(tokens)


with open(output, "w") as f:
    for m in misspelled:
        corrected = spell.correction(m)
        text = text.replace(m, corrected)
        # f.write(c.replace('clafliques', 'classiques'))
    f.write(text)
    

In [9]:
spell = SpellChecker(language='fr')
spell.split_words("l’estât")

['l', 'estât']

In [ ]:
import re
entry = "2.txt"


text = open(entry).read()

r1 = re.findall(r"([lL]’\w+|[dD]’\w+|[sS]’\w+|[qQ]u’\w+|[cC]’\w+|[nN]’\w+|[jJ]’\w+|[Ll]orfqu’\w+|eft|http.*)",text)
print(r1)

In [11]:
from spellchecker import SpellChecker
spell = SpellChecker(language='fr')
c = spell.correction('c’estoit')
c = spell.correction('seigneurie')
c = spell.correction('faict')
c = spell.correction('marquisat')
c = spell.correction('MÉMOÎRES')
c = spell.correction('l’estât faict bijour')
print(c)

l’estât faict bijour


In [ ]:
for m in misspelled:
    print(m)

In [ ]:
from lxml import etree
import re

tree = etree.parse('test.xml')
root = tree.getroot()

with open('test_out.txt', 'w') as f:
    for elem in root.iter('*'):
        if elem.text is not None:
            text = elem.text.strip()
            if text: 
                text = text.replace("\n", "")
                text = re.sub(" +", " ", text)
                print(text)
                f.write(text + "\n")

### test 1 : OK<br>
### test 2 : NOK

In [ ]:
# importer les librairies nécessaires
import os, re, glob
from spellchecker import SpellChecker


# ignorer les fichiers cachés dans le directoire avec les docs d'entrée (p. ex. le '._5419000_r.xml') 
def listdir_nohidden(path):
    return glob.glob(os.path.join(path, '*'))


# spécifier les docs d'entrée (échantillon) à partir desquels les sorties corrigées seront générées
directory_in = listdir_nohidden("./sample_in/")
directory_out = listdir_nohidden("./sample_out/")
 

# enlever les balises xml
for file_in in directory_in:
    #file_in = os.path.basename(file_in)
    text = re.sub('<[^<]+>', "", open(file_in).read())
    text = open(file_in).read()
    # enlever les espaces multiples
    #text = re.sub('\s+',' ', open(entry).read())


    # définir le correcteur d'orthographe français 
    # pyspellchecker
    content_list = text.split()
    spell = SpellChecker(language='fr')
    misspelled = spell.unknown(content_list)


    # ajouter le mot '&amp;' dans le dictionnaire des mots corrects pour ne pas le corriger en 'camp'
    spell.word_frequency.load_words(['&amp;'])
    # spell.known(['&amp;'])  # le mot {'&amp;'} est désormais dans le dictionnaire des mots corrects

    
    # corriger les mots incorrects
    for m in misspelled:
        corrected = spell.correction(m)
        # print(corrected)
        

    # ouvrir les nouveaux fichiers qui seront corrigés
    file_in = os.path.basename(file_in)
    file_out = os.path.splitext(file_in)[0]
    file_out = './sample_out/{}_out.xml'.format(file_out)
    
 
    # corriger directement dans le doc d'entrée
    with open(file_out, 'w') as f:
        for m in misspelled:
            corrected = spell.correction(m)
            text = text.replace(m, corrected)
            # f.write(c.replace('clafliques', 'classiques'))
        f.write(text)

In [ ]:
from spellchecker import SpellChecker
spell = SpellChecker(language='fr')
a = spell.correction('AnnotateurPluquet')
print(a)

In [ ]:
import re
from spellchecker import SpellChecker

entry = "test_out.xml"
output = "test_out2.xml"


# enlever les balises xml
#text = re.sub('<[^<]+>', "", open(entry).read())


# enlever les espaces multiples
# text = re.sub('\s+',' ', open(entry).read())


# activer le correcteur d'orthographe français
content_list = text.split()
spell = SpellChecker(language='fr')
misspelled = spell.unknown(content_list)


# ajouter le mot '&amp;' dans le dictionnaire des mots corrects pour ne pas le corriger en 'camp'
spell.word_frequency.load_words(['&amp;'])
spell.known(['&amp;'])  # le mot {'&amp;'} est désormais dans le dictionnaire des mots corrects


# corriger les mots incorrects
# for m in misspelled:
#     corrected = spell.correction(m)
    # print(corrected)

with open(output, "w") as f:
    for m in misspelled:
        corrected = spell.correction(m)
        text = text.replace(m, corrected)
        # f.write(c.replace('clafliques', 'classiques'))
    f.write(text)

In [ ]:
from HTMLParser import HTMLParser

class extract_text(HTMLParser):
    def handle_data(self,data):
        if data.strip():
            print data

parser = extract_text()
parser.feed(doc)

In [ ]:
for tag in soup.findAll('p'):
    print(tag.string)

### Correction d'un seul fichier

In [ ]:
import re
from spellchecker import SpellChecker

entry = "test.xml"
output = "test_out.xml"


# enlever les balises xml
text = re.sub('<[^<]+>', "", open(entry).read())


# enlever les espaces multiples
# text = re.sub('\s+',' ', open(entry).read())


# activer le correcteur d'orthographe français
content_list = text.split()
spell = SpellChecker(language='fr')
misspelled = spell.unknown(content_list)


# ajouter le mot '&amp;' dans le dictionnaire des mots corrects pour ne pas le corriger en 'camp'
spell.word_frequency.load_words(['&amp;'])
spell.known(['&amp;'])  # le mot {'&amp;'} est désormais dans le dictionnaire des mots corrects


# corriger les mots incorrects
# for m in misspelled:
#     corrected = spell.correction(m)
    # print(corrected)

with open(output, "w") as f:
    for m in misspelled:
        corrected = spell.correction(m)
        text = text.replace(m, corrected)
        # f.write(c.replace('clafliques', 'classiques'))
    f.write(text)

In [ ]:
import re
from spellchecker import SpellChecker

entry = "test_pt_1.xml"
output = "test_pt_1_out.xml"


# enlever les balises xml
text = re.sub('<[^<]+>', "", open(entry).read())


# enlever les espaces multiples
# text = re.sub('\s+',' ', open(entry).read())


# activer le correcteur d'orthographe français
content_list = text.split()
spell = SpellChecker(language='fr')
misspelled = spell.unknown(content_list)


# ajouter le mot '&amp;' dans le dictionnaire des mots corrects pour ne pas le corriger en 'camp'
spell.word_frequency.load_words(['&amp;'])
spell.known(['&amp;'])  # le mot {'&amp;'} est désormais dans le dictionnaire des mots corrects


# corriger les mots incorrects
# for m in misspelled:
#     corrected = spell.correction(m)
    # print(corrected)

with open(output, "w") as f:
    for m in misspelled:
        corrected = spell.correction(m)
        text = text.replace(m, corrected)
        # f.write(c.replace('clafliques', 'classiques'))
    f.write(text)

### Correction par lots (essai)

In [ ]:
import re
from spellchecker import SpellChecker

entry = "5419000_r.xml"
output = "5419000_r_out.xml"


# enlever les balises xml
text = re.sub('<[^<]+>', "", open(entry).read())


# enlever les espaces multiples
# text = re.sub('\s+',' ', open(entry).read())


# activer le correcteur d'orthographe français
content_list = text.split()
spell = SpellChecker(language='fr')
misspelled = spell.unknown(content_list)


# ajouter le mot '&amp;' dans le dictionnaire des mots corrects pour ne pas le corriger en 'camp'
# spell.word_frequency.load_words(['&amp;'])
# spell.known(['&amp;'])  # le mot {'&amp;'} est désormais dans le dictionnaire des mots corrects


# corriger les mots incorrects
for m in misspelled:
    corrected = spell.correction(m)

with open(output, "w") as f:
    #for m in misspelled:
        #corrected = spell.correction(m)
    text = text.replace(m, corrected)
        # f.write(c.replace('clafliques', 'classiques'))
    #f.write(text)
    print(text)

In [ ]:
spell.correction('Confucius')

In [ ]:
import re
from spellchecker import SpellChecker

entry = "5419000_r.xml"
output = "5419000_out.xml"

# enlever les espaces multiples
# text = re.sub('\s+',' ', open(entry).read())


# enlever les balises xml
text = re.sub('<[^<]+>', "", open(entry).read())


# activer le correcteur d'orthographe français
content_list = text.split()
spell = SpellChecker(language='fr')


# ajouter le mot '&amp;' dans le dictionnaire des mots corrects pour ne pas le corriger en 'camp'
spell.word_frequency.load_words(['&amp;'])
# spell.known(['&amp;'])  # le mot {'&amp;'} est désormais dans le dictionnaire des mots corrects


misspelled = spell.unknown(content_list)


# corriger les mots incorrects
for m in misspelled:
    corrected = spell.correction(m)
    # print(corrected)

with open(output, "w") as f:
    for m in misspelled:
        corrected = spell.correction(m)
        text = text.replace(m, corrected)
        # f.write(c.replace('clafliques', 'classiques'))
    f.write(text)

### Correction par lots

In [ ]:
# importer les librairies nécessaires
import os, re, glob
from spellchecker import SpellChecker


# ignorer les fichiers cachés dans le directoire avec les docs d'entrée (p. ex. le '._5419000_r.xml') 
def listdir_nohidden(path):
    return glob.glob(os.path.join(path, '*'))


# spécifier les docs d'entrée (échantillon) à partir desquels les sorties corrigées seront générées
directory_in = listdir_nohidden("./sample_in/")
directory_out = listdir_nohidden("./sample_out/")
 

# enlever les balises xml
for file_in in directory_in:
    #file_in = os.path.basename(file_in)
    text = re.sub('<[^<]+>', "", open(file_in).read())


    # définir le correcteur d'orthographe français 
    # pyspellchecker
    content_list = text.split()
    spell = SpellChecker(language='fr')
    misspelled = spell.unknown(content_list)


    # ajouter le mot '&amp;' dans le dictionnaire des mots corrects pour ne pas le corriger en 'camp'
    spell.word_frequency.load_words(['&amp;'])
    # spell.known(['&amp;'])  # le mot {'&amp;'} est désormais dans le dictionnaire des mots corrects

    
    # corriger les mots incorrects
    for m in misspelled:
        corrected = spell.correction(m)
        # print(corrected)
        

    # ouvrir les nouveaux fichiers qui seront corrigés
    file_in = os.path.basename(file_in)
    file_out = os.path.splitext(file_in)[0]
    file_out = './sample_out/{}_out.xml'.format(file_out)
    
 
    # corriger directement dans le doc d'entrée
    with open(file_out, 'w') as f:
        for m in misspelled:
            corrected = spell.correction(m)
            text = text.replace(m, corrected)
            # f.write(c.replace('clafliques', 'classiques'))
        f.write(text)

In [ ]:
# importer les librairies nécessaires
import os, re, glob
from spellchecker import SpellChecker


# ignorer les fichiers cachés dans le directoire avec les docs d'entrée (p. ex. le '._5419000_r.xml') 
def listdir_nohidden(path):
    return glob.glob(os.path.join(path, '*'))


# spécifier les docs d'entrée (échantillon) à partir desquels les sorties corrigées seront générées
directory_in = listdir_nohidden("./sample_in/")
directory_out = listdir_nohidden("./sample_out/")
 

# enlever les balises xml
for file_in in directory_in:
    #file_in = os.path.basename(file_in)
    text = re.sub('<[^<]+>', "", open(file_in).read())


    # définir le correcteur d'orthographe français 
    # pyspellchecker
    content_list = text.split()
    spell = SpellChecker(language='fr')
    misspelled = spell.unknown(content_list)


    # ajouter le mot '&amp;' dans le dictionnaire des mots corrects pour ne pas le corriger en 'camp'
    spell.word_frequency.load_words(['&amp;'])
    # spell.known(['&amp;'])  # le mot {'&amp;'} est désormais dans le dictionnaire des mots corrects

    
    # corriger les mots incorrects
    for m in misspelled:
        corrected = spell.correction(m)
        # print(corrected)
        

    # ouvrir les nouveaux fichiers qui seront corrigés
    file_in = os.path.basename(file_in)
    file_out = os.path.splitext(file_in)[0]
    file_out = './sample_out/{}_out.xml'.format(file_out)
    file_csv = './sample_out/errors_out.csv'
 
    # corriger directement dans le doc d'entrée
    with open(file_out, 'w') as f:
        for m in misspelled:
            corrected = spell.correction(m)
            text = text.replace(m, corrected)
            # f.write(c.replace('clafliques', 'classiques'))
        f.write(text)
        
    

    # calculer la fréquence des erreurs
    for c in content_list:
        c_freq = str(content_list.count(c))
        # print(c,c_freq)

    
# enregistrer les erreurs, leurs corrections et le nombre d'occurrence d'erreurs dans un tableur .csv
file_csv = './sample_out/errors_out.csv'

with open(file_csv, 'w') as f_csv:
    writer = csv.writer(f_csv)
    writer.writerow(["Erreur"'\t' "Correction"'\t' "Fréquence"'\t'])
    for m in misspelled:
        # misspelled = spell.unknown(content_list)
        # print(misspelled) # imprimer les tokens mal orthographiés
        # for word in misspelled:
        f_csv.write(m+'\t' + spell.correction(m)+'\t' + ' \n')

In [ ]:
    for c in content_list:
        c_freq = str(content_list.count(c))
        print(c,c_freq)

In [ ]:
    for m in misspelled:
       # misspelled = spell.unknown(content_list)
        print(m)

In [ ]:
# calculer la fréquence des erreurs
for c in content_list:
    if c in misspelled:
        c_freq = str(content_list.count(c))
    print(c_freq, c)

### Extraction des erreurs dans un tableur `.csv`

In [ ]:
# importer les librairies nécessaires
import regex as re
import csv
from spellchecker import SpellChecker


# # spécifier les docs d'entrés à corriger et ceux de sortie qui seront corrigés
# entry = "sample.xml"
# output = "sample_out.xml"

# ignorer les fichiers cachés dans le directoire avec les docs d'entrée (p. ex. le '._5419000_r.xml') 
def listdir_nohidden(path):
    return glob.glob(os.path.join(path, '*'))


# spécifier les docs d'entrée (échantillon) à partir desquels les sorties corrigées seront générées
directory_in = listdir_nohidden("./sample_in/")
directory_out = listdir_nohidden("./sample_out/")


# enlever les balises XML
text = re.sub('<[^<]+>', "", open(entry).read()) 
with open(output, "w") as f:
    f.write(text)
    

# enlever les espaces dans le fichier XML    
text = re.sub('\s+',' ', open(output).read())

    
# enlever les signes de ponctuation sauf les apostrophes, les points d’interrogations, les parenthèses et les tirets
text = re.sub(r"""[!^\.\,\!\*]+""",'',text)
with open(output, "w") as f:
    f.write(text)  
    
    
# récupérer la liste des tokens   
my_file = open(output, "r")
content = my_file.read()
# print(content) # imprimer le contenu

content_list = content.split(" ")
my_file.close()
# print(content_list) # imprimer la liste des tokens


#  extraire les tokens mal orthographiés et leurs corrections
fout = open("corrections.csv", "wt")
spell = SpellChecker(language='fr')
misspelled = spell.unknown(content_list)
writer = csv.writer(fout)
writer.writerow(["Erreur"'\t' "Correction"'\t' "Fréquence"'\t'])
# print(misspelled) # imprimer les tokens mal orthographiés
for word in misspelled:
	fout.write(word+'\t' + spell.correction(word)+'\t' + ' \n')
	

fout.close()

### Ça marche sur le fichier élargi

In [ ]:
import re
from spellchecker import SpellChecker

entry = "test.xml"
output = "test_out_2.xml"

# remove multiple spaces
# text = re.sub('\s+',' ', open(entry).read())

# remove the xml tags
text = re.sub('<[^<]+>', "", open(entry).read())

# set the French spell checker

content_list = text.split()
spell = SpellChecker(language='fr')
misspelled = spell.unknown(content_list)

for m in misspelled:
    corrected = spell.correction(m)
    # print(corrected)

with open(output, "w") as f:
    for m in misspelled:
        corrected = spell.correction(m)
        text = text.replace(m, spell.correction(m))
        # f.write(c.replace('clafliques', 'classiques'))
    f.write(text)

### Cela fonctione-t-il sur le fichier complet ? <br>
Non, on a toujours les mêmes erreurs.<br>
`Confucius` > `Confucrus`, `Observations` > `Obsservaitens`...

In [ ]:
import re
from spellchecker import SpellChecker

entry = "5419000_r.xml"
output = "5419000_r_out_2.xml"

# remove multiple spaces
# text = re.sub('\s+',' ', open(entry).read())

# remove the xml tags
text = re.sub('<[^<]+>', "", open(entry).read())

# set the French spell checker

content_list = text.split()
spell = SpellChecker(language='fr')
misspelled = spell.unknown(content_list)

for m in misspelled:
    corrected = spell.correction(m)
    # print(corrected)

with open(output, "w") as f:
    for m in misspelled:
        corrected = spell.correction(m)
        text = text.replace(m, spell.correction(m))
        # f.write(c.replace('clafliques', 'classiques'))
    f.write(text)

### OCR-NE

In [ ]:
import re
from spellchecker import SpellChecker

entry = "test.xml"
output = "test_out.xml"

# enlever les espaces multiples
# text = re.sub('\s+',' ', open(entry).read())


# enlever les balises xml
text = re.sub('<[^<]+>', "", open(entry).read())


# activer le correcteur d'orthographe français
content_list = text.split()
spell = SpellChecker(language='fr')
misspelled = spell.unknown(content_list)


# ajouter le mot '&amp;' dans le dictionnaire des mots corrects pour ne pas le corriger en 'camp'
spell.word_frequency.load_words(['&amp;'])
# spell.known(['&amp;'])  # le mot {'&amp;'} est désormais dans le dictionnaire des mots corrects


# corriger les mots incorrects
for m in misspelled:
    corrected = spell.correction(m)
    # print(corrected)

with open(output, "w") as f:
    for m in misspelled:
        corrected = spell.correction(m)
        text = text.replace(m, corrected)
        # f.write(c.replace('clafliques', 'classiques'))
    f.write(text)